In [106]:
import pandas as pd
import re
import numpy as np

## ALGOLIA
from algoliasearch.search_client import SearchClient
client = SearchClient.create("XD7RZG62BF","21e90530eb95b823eaa2d88154123b2b")
algoliaIndex = client.init_index('boot_camp')

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

fileOut = "fuzzy_educ_boots.xlsx"

In [48]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [49]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfe = df[df["education"].notna()] #DATAFRAME DE EDUCATION
dfe = dfe.reset_index(drop = True) #REINICIAR INDICE

dfc = df[df["certifications"].notna()] #DATAFRAME DE CERTIFICACIÓN

In [50]:
dfe.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [51]:
dfe.shape

(2566, 29)

In [52]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE EDUCACIÓN

In [53]:
#CONVERTIR EN STRING
dfe["education"] =  dfe["education"].str.lower()

In [54]:
#CONVERTIR EN DICCIONARIOS LA EDUCACIÓN
L = []

for i in dfe["education"]:
    l = str(i).split("},") #convertir en string la formación del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for formacion in l: #para cada formación
        formacion = formacion.replace("[", "").replace("]", "") + "}" #separelas y conviertalas en diccionarios
        k.append(formacion) #añadalas a la lista de formación
    L.append(k) #añada la lista de formación para individuo

#VER UN EJEMPLO: Esta es la última formación del primer individuo desordenada en diccionario pero como un string   
L[0][0]

"{'education_org': 'university of oxford', 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user', 'end': '2013', 'start': '2013'}"

In [55]:
K = [] #LISTA D ELISTAS EN DONDE SE ALMACENARAN LAS FORMACIONES A MANERA DE DICCIONARIO PARA CADA INDIVIDUO

for individuo in L: #para cada individuo

    educacion_ind = [] #creo una lista de sus educaciones

    for ins in individuo: # para cada cada formación dentro de las formaciones de cada individuos
        ins = ins.replace('"', "'")

        c = {}

        #CADA CAMPO SE EXTRAE POR MEDIO DE RegEx, COMUNES A TODOS LAS FORMACIONES STR DE LOS INDIVIDUOS EN EL DATASET
        #CADA CAMPO SE EXTRAE EN CASO DE QUE EXISTA DE LO CONTRARIO NO

        #EXTRAER EDUCACIÓN
        if ( len(re.findall("'education_org': '.*?'", ins)) ) > 0:
            c["education"] =  re.findall("'education_org': '.*?'", ins)[0].replace("'education_org': ", "").replace("'", "")

        #EXTRAER DEGREE
        if ( len(re.findall("'degrees': '.*?'", ins)) ) > 0:
            c["degrees"] =  re.findall("'degrees': '.*?'", ins)[0].replace("'degrees': ", "").replace("'", "")

        #EXTRAER SUMMARY
        if ( len(re.findall("'summary': '.*?'", ins)) ) > 0:
            c["summary"] =  re.findall("'summary': '.*?'", ins)[0].replace("'summary': ", "").replace("'", "")


        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
K[0][0]

{'education': 'university of oxford',
 'degrees': 'university of oxford – oxford royale academy, english and culture, common european framework of reference: profiecient user',
 'start': '2013',
 'end': '2013'}

In [56]:
#CREAR COLUMNA LIMPIA DE EDUCACIÓN
dfe["educacion"] = K

In [73]:
#VER UN EJEMPLO: Esta es la última formación del primer individuo ordenada en un diccionario
dfe.educacion[0][1]

{'education': 'vrije universiteit amsterdam',
 'degrees': 'master of science (msc), policy, communication and organization',
 'summary': 'afstudeerrichting: organisatie en omgeving\\n afstudeeronderzoek bij bank x\\n onderzoek gedaan naar de zin- en betekenisgeving die leidinggevenden van afdeling x toekennen aan de (nieuwe) positionering van bank x en de wijze van doorvertaling naar hun medewerkers.\\n\\n begeleid door prof. dr. m. veenswijk: hoogleraar management van cultuurverandering en afdelingshoofd organisatiewetenschappen aan de vrije universiteit amsterdam\\n activities and societies: lid van faculteitsvereniging eos alumnus, alumni faculteitsvereniging eos alumnus, alumni faculteit der sociale wetenschappen',
 'start': '2012',
 'end': '2013'}

In [90]:
## se agregan a Algolia
## just necesary info
#total lines
total = dfe.shape[0]
for index in range(0,total):
    educationsIND = dfe.educacion[index]
    totalTempEdu = len(educationsIND)
    ##plain string for education
    educationString = []
    for edu in range(0,totalTempEdu):
        eduObj = dfe.educacion[index][edu]
        if "education" in eduObj:
            educationString.append(eduObj["education"])
        else:
            print(eduObj)
    algoliaObje = {}
    algoliaObje["objectID"] = index
    algoliaObje["education"] = educationString
    print(algoliaObje)
    algoliaIndex.save_object(algoliaObje)
    
    #
    #print(row.id)
    


{'objectID': 1868, 'education': ['faculty of management studies - university of delhi', 'indian institute of technology, madras', 'st. george', 'indian institute of technology, madras']}
{'objectID': 1869, 'education': ['washtenaw community college', 'wtmc']}
{'objectID': 1870, 'education': ['university of maryland college park', 'bangladesh university of engineering and technology', 'notre dame college, dhaka', 'government laboratory high school, dhaka']}
{'objectID': 1871, 'education': ['front sight firearms training institute']}
{'objectID': 1872, 'education': ['universidade do oeste paulista', 'universidade do oeste paulista', 'universidade do oeste paulista']}
{'objectID': 1873, 'education': ['medical high school in belgrade serbia', 'univerzitet u beogradu / university of belgrade']}
{'objectID': 1874, 'education': ['unoeste']}
{'objectID': 1875, 'education': ['argosy university honolulu', 'argosy university honolulu', 'regent university', 'university of michigan - stephen m. ro

AlgoliaUnreachableHostException: Unreachable hosts

In [115]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc',
 'BICTIA',
 'Talento Digital']

In [13]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion'
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

 #VERDE: MÁS GRANDES DE LA, #AMARILLOS: MEDIANOS
 #EN PRINCIPIOS LOS VERDES SON LOS QUE DEBERIAN APARECER MÁS EN LOS PERFILES, EN PRINCIPIO.


 #BASE: LINKED-IN: + DIFICIL PARA HACER SCRAPING. EMPRESAS VENDEN DATOS (OBTUVIERON HACIENDO SCRAPING), PERFILES EGRESADOS DE BOOTCAMPS DE LISTA DE BOOTCAMPS EN LA. 3 RONDAS DE INTENTOS Y MANDAN MUESTRA PEQUEÑA. SUGIEREN MUCHOS DE LOS CAMBIOS.abs

 #NEGOCIACIÓN, PERFILES Y CUÁNTO CUESTA. MAX 8MIL. CADA FALSO POSITIVO ES UN PROBLEMA. 
 # LLEGABAN A MIL, PARECIA QUE FP NO ERAN PROBLEMA: MEJOR ESTRATEGIA PALABRAS GENERALES Y TRAEMOS 5MIL PERFILES Y DENTRO DE ESA DEBEN DE ESTAR TODOS ESTUDIANTES DE PERFILES PROFESIONALES DE BOOTCAMPS.
 # EGRESADOS EN LATIRNOAMERICA DEBERÍAN SER MUCHO MÁS DE 8MIL, EJ: DIGITAL HOUSE HABLAN DE 100MIL EGRESADOS

 #NO REPRESENTATIVIDAD: MUCHOS EGRESADOS POCA MUESTRA, NO TODOS TIENEN PERFIL DE LINKED-IN

 #PRIORIDAD

In [119]:
#LOOP PARA BUSCAR SI EL INDIVIDUO TIENE FORMACIÓN EN ALGÚN BOOTCAMP
c = []
## se utiliza los nombres normales
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfe.educacion)): #PARA CADA INDIVIDUO
        for h in range(len(dfe.educacion[i])): #DENTRO LA EDUCACIÓN DE CADA INDIVIDUO
            if "education" in dfe.educacion[i][h]: #SI TIENE EL CAMPO EDUCACIÓN
                ## se hace un aproach a fuzzy
                #A = re.findall(boots[boot], dfe.educacion[i][h]["education"]) #REVISA SI ESTÁ EN UN BOOTCAMP
                #f len(A) > 0 : #SI ESA REVISIÓN ES NO VACÍA
                ## se normaliza la entrada
                bootCAMP = lista_boots[boot]
                bootCAMP = bootCAMP.lower()
                bootCAMP = unidecode(bootCAMP)
                norEdu =  unidecode(dfe.educacion[i][h]["education"])
                ## se hace un aproach a fuzzy
                wordLen = len(dfe.educacion[i][h]["education"])
                ratio = fuzz.partial_ratio(bootCAMP,norEdu)
                #print(ratio)
                if ratio>85 and wordLen >=5:
                    c.append((dfe.id[i], lista_boots[boot], lista_boots[boot], dfe.educacion[i][h]['education'],ratio))
                    # c.append((dfe.id[i], h)) #AÑADE EL ID DE ESE INDIVIDUO Y EL LUGAR DONDE ESTÁ ESA EDUCACIÓN
                    print(f"{dfe['job_title'][i]}---{ratio}---{lista_boots[boot]}--{dfe.educacion[i][h]['education']} \n \n")


            #LOOP SIN INDICE
            # for edu in dfe.educacion[i]:
            # if "education" in edu:
            #     # print(boot, edu["education"])
            #     A = re.findall(boot, edu["education"])
            #     if len(A) > 0:
            #         c.append(A)
            #         m.append(boot)
            #         print(f" {dfe['job_title'][i]} -- {A} -- {boot} -- {edu['education']} \n \n")
            #         # print(f"{A} -- {boot} -- {edu['education']}")

--dev f 
 

Product Manager---100---Digital House--digital house 
 

Ejecutivo de cuentas---100---Digital House--digital house 
 

Social Media Intelligence & Marketing Research---100---Digital House--digital house 
 

Administración---100---Digital House--digital house 
 

English > Spanish Subtitler---100---Digital House--digital house 
 

Agile Coach---100---Digital House--digital house 
 

nan---100---Digital House--digital house 
 

Digital Acquisition---100---Digital House--digital house 
 

Ejecutivo de Ventas---100---Digital House--digital house 
 

Sales Team Lead, EU Marketplace Expansion---100---Digital House--digital house 
 

International Consultant---100---Digital House--digital house 
 

District Manager---100---Digital House--digital house 
 

Responsable de caja---100---Digital House--digital house 
 

Customer Success Manager---100---Digital House--digital house 
 

Customer Success Manager---100---Digital House--digital house 
 

Gerente de Calidad---100---Digital H

In [120]:
#CREAR DATAFRAME

c = list(set(c))
educ_boots = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp', "educ_title","ratio"])

In [121]:
#HACER MERGE CON EL RESTO DE COLUMNAS
educ_boots = pd.merge(educ_boots, dfe, "left", "id")
educ_boots.to_excel(fileOut, index = False)

In [ ]:
df_educ 